**need some %run command and uncomment the model load before submitting**

In [ ]:
print("---------run baseline")

In [ ]:
import sys,json,re,string
from typing import Callable, Dict, List, Set, Tuple
from elasticsearch import Elasticsearch
import gensim.downloader as api
import gensim
import datetime
import csv
sys.path.insert(0, 'extract_features')
from extract_No1to5_features import extract_features_1to5
from extract_No11to12_features import extract_features_11to12
from extract_No13to15_features import TypeTaxonomy, extract_features_13to15
from extract_No16_feature import extract_features_16
from extract_No17to19_features import get_analyze,extract_features_17to19
from extract_No20to22_features import extract_features_20to22
from extract_No23to25_features_optimized import extract_features_23to25

In [ ]:
# Reduce logging level.
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.WARNING)
logging.basicConfig(level=logging.ERROR)

logging.getLogger("elasticsearch").disabled = True
# es_log = logging.getLogger("elasticsearch")
# es_log.setLevel(logging.CRITICAL)
es= Elasticsearch(timeout=600)
es.info()

In [ ]:
#%run extract_features/extract_No11to12_features

**for my computer , need the following enviroment to run this script**
<br>
/c/Users/junec/anaconda3/python
<br>
/c/Users/junec/anaconda3/Scripts/pip

In [ ]:
!which python
!which pip

In [ ]:
#load files
filepath="data/DBpedia_map_type_entities.json"
with open(filepath, 'r', encoding='utf-8') as file:
    DBpedia_map_type_entities = json.load(file)
    
filepath="data/training_types.json"
with open(filepath,encoding='utf-8') as json_file:
    training_map_type_questions = json.load(json_file)

filepath="data/ElasticSearch_map_type_docID.json"
with open(filepath, 'r',encoding='utf-8') as f:
    docID_DBOtype_dict = json.load(f)

filepath="../smart-dataset/datasets/DBpedia/smarttask_dbpedia_train.json"
with open(filepath, 'r') as f:
    smarttask_dbpedia_train = json.load(f)

typeobj=TypeTaxonomy("data/dbpedia_types.tsv")

In [ ]:
# type_list=list(typeobj._types.keys())
# type_list[1:]
# len(type_list[1:]),len(smarttask_dbpedia_train) ,smarttask_dbpedia_train[1]

In [ ]:
# dp_type="dbo:MilitaryPerson"
# question="Does the shelf life of spinach equal 8?"

# dp_type="dbo:MusicFestival"
# question="When was Bibi Andersson music festival married to Per Ahlmark very green?"

# extract_features_1to5(DBpedia_map_type_entities,dp_type,question,es)
# features_1to5=extract_features_1to5(DBpedia_map_type_entities,dp_type,question,es)
# extract_features_11to12(dp_type, question,docID_DBOtype_dict,es)
# extract_features_13to15(typeobj,dp_type)
# print(extract_features_16(DBpedia_map_type_entities,dp_type))

# extract_features_17to19(training_map_type_questions,dp_type,question)
# extract_features_20to22(dp_type,question)
# extract_features_23to25(model_loaded,dp_type, question, mode="Euclidean")
# extract_features_23to25(model_loaded,dp_type, question, mode="similarities")

In [ ]:
def preprocess(doc: str) -> str:
    """Preprocesses text to prepare it for feature extraction.

    Args:
        doc: String comprising the unprocessed contents of some email file.

    Returns:
        String comprising the corresponding preprocessed text.
    """
    re_html = re.compile("<[^>]+>")
    doc = re_html.sub(" ", doc)
    #remove pure digits 
    doc=re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b","",doc)
    # Replace punctuation marks (including hyphens) with spaces.
    for c in string.punctuation:
        doc = doc.replace(c, " ")
    #return doc.lower()
    return doc

In [ ]:
def extract_features_baseline(
    question:str,
    dp_type: str,
    DBpedia_map_type_entities:Dict,
    docID_DBOtype_dict:Dict,
    es: Elasticsearch,
 
) -> List[float]:
    """Extracts features of a query and document pair.

        Args:
            query: string.
            dp_type: DBO type.
            es: Elasticsearch object instance.

        Returns:
            List of extracted feature values in a fixed order.
    """
    features_1to5 = extract_features_1to5(DBpedia_map_type_entities,dp_type,question,es)
    feature_vect = list(features_1to5.values())

    features_11to12 = extract_features_11to12(dp_type, question,docID_DBOtype_dict,es)
    feature_vect.extend(list(features_11to12.values()))
    
    return feature_vect

In [ ]:
#with first path filter
def prepare_training_data_baseline( smarttask_dbpedia_train:List[Dict],
                            DBpedia_map_type_entities:Dict[str,List],
                            docID_DBOtype_dict:Dict[str,str],
                            es: Elasticsearch
                            )-> Tuple[List[List[float]], List[int]]:
    """
    """
#     X_train=[]
#     y_train=[]
    all_DBOtype=list(typeobj._types.keys())
    #remove the first type:"thing"
    all_DBOtype=all_DBOtype[1:]
    
    count=0
    j=0
#     file_X = open("data/X_train_baseline.csv", 'w', newline='')
#     writer_X = csv.writer(file_X)
#     file_y = open("data/y_train_baseline.csv", 'w', newline='')
#     writer_y = csv.writer(file_X)
    with open("data/for_training_baseline.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)


        for entry in smarttask_dbpedia_train:
            if j%100==0:
                print("--------------------j:",j)

            j+=1
            if entry['question']==None:
                continue

            question_processed=preprocess(entry['question'])

            if entry['category']=='resource':
                if count%20==0:
                    print("--------------------count:",count)
                    print("-----------------",datetime.datetime.now())
                count+=1
                print("question_processed:",question_processed)
                for DBOtype in entry['type']: 
                    try:
                        features=extract_features_baseline(question_processed,
                                                            DBOtype,
                                                            DBpedia_map_type_entities,
                                                            docID_DBOtype_dict,
                                                            es)
                    except BaseException as err:
                        print("------------error for type:",DBOtype,entry['question'])
                        print(f"Unexpected {err}, {type(err)}")  
                        raise
#                     X_train.append(features)
#                     y_train.append(1)
                    writer.writerow(features)
                    writer.writerow([1])


    #                 writer_X.writerow(features)
    #                 writer_y.writerow(y_train)

                #deal with top 100 documents
                hits = es.search(
                    index="dbpdiea_type_centric", q=question_processed, _source=True, size=30
                )["hits"]["hits"]
                rank_list= [hit['_source']["type"] for hit in hits]
                #print(rank_list)

                for DBOtype in rank_list:
                    if DBOtype not in entry['type']:
                        try:
                            features=extract_features_baseline(question_processed,
                                                                DBOtype,
                                                                DBpedia_map_type_entities,
                                                                docID_DBOtype_dict,
                                                                es)
                        except BaseException as err:
                            print("------------error for type:",DBOtype,entry['question'])
                            print(f"Unexpected {err}, {type(err)}")  
                            raise
                        #print(features)
#                         X_train.append(features)
#                         y_train.append(0)
                        writer.writerow(features)
                        writer.writerow([0])
#                         writer_X.writerow(features)
#                         writer_y.writerow(y_train)

    #                 if count%100000==0:
    #                     print(f'--------------{count} is finished')
    #                     if len(X_train)%500==0:
    #                         print("-----------------",datetime.datetime.now())
    #                         print(f'--------------length of X_train:{len(X_train)} y_train: {len(y_train)}')
        
    print(f'{count} features has been saved,{j} questions have been processed')
#     file_X.close()
#     file_y.close()
    return None



prepare_training_data_baseline(smarttask_dbpedia_train,
                                        DBpedia_map_type_entities,
                                        docID_DBOtype_dict,
                                        es) 

In [ ]:
# #no first pass filter
# def prepare_training_data_baseline( smarttask_dbpedia_train:List[Dict],
#                             DBpedia_map_type_entities:Dict[str,List],
#                             docID_DBOtype_dict:Dict[str,str],
#                             es: Elasticsearch
#                             )-> Tuple[List[List[float]], List[int]]:
#     """
#     """
#     X_train=[]
#     y_train=[]
#     all_DBOtype=list(typeobj._types.keys())
#     #remove the first type:"thing"
#     all_DBOtype=all_DBOtype[1:]
    
#     count=0
#     j=0
#     file_X = open("data/X_train_baseline.csv", 'w', newline='')
#     writer_X = csv.writer(file_X)
#     file_y = open("data/y_train_baseline.csv", 'w', newline='')
#     writer_y = csv.writer(file_X)
# #     with open("data/baseline_features.csv", 'w', newline='') as csvfile:
# #         writer = csv.writer(csvfile)


#     for entry in smarttask_dbpedia_train:
# #         if j%int(len(smarttask_dbpedia_train)/100)==0:
# #             print(f'--------------{j/int(len(smarttask_dbpedia_train)/100)}% is finished')
#         if j%100==0:
#             print("--------------------j:",j)

#         j+=1
#         if entry['question']==None:
#             continue
            
#         question_processed=preprocess(entry['question'])
        
#         if entry['category']=='resource':
#             for DBOtype in all_DBOtype: 
#                 try:
#                     features=extract_features_baseline(question_processed,
#                                                         DBOtype,
#                                                         DBpedia_map_type_entities,
#                                                         docID_DBOtype_dict,
#                                                         es)
#                 except BaseException as err:
#                     print("------------error for type:",DBOtype,entry['question'])
#                     print(f"Unexpected {err}, {type(err)}")  
#                     raise
#                 X_train.append(features)
#                 count+=1
#                 #print(f'--------------length of X_train:{len(X_train)} y_train: {len(y_train)}')
#                 if DBOtype in entry['type']: 
#                     y_train.append(1)
#                 else:
#                     y_train.append(0)


#                 writer_X.writerow(features)
#                 writer_y.writerow(y_train)

#                 if count%100000==0:
#                     print(f'--------------{count} is finished')
#                 if len(X_train)%500==0:
#                     print("-----------------",datetime.datetime.now())
#                     print(f'--------------length of X_train:{len(X_train)} y_train: {len(y_train)}')
        
#     print(f'{count} features has been saved,{j} questions have been processed')
#     file_X.close()
#     file_y.close()
#     return X_train,y_train



# X_train,y_train=prepare_training_data_baseline(smarttask_dbpedia_train,
#                                         DBpedia_map_type_entities,
#                                         docID_DBOtype_dict,
#                                         es) 

In [ ]:
# with open("data/X_train_baseline.json", 'w') as f:
#   json.dump(X_train, f)
# with open("data/y_train_baseline.json", 'w') as f:
#   json.dump(y_train, f)
# print("X_train,y_train are saved as json files")

In [ ]:
file = open("data/baseline_features.csv")
csvreader = csv.reader(file)
rows = []
for row in csvreader:
        rows.append(row)
rows

type(rows[3])
rows[3]
